In [143]:
import pandas as pd

# Replace the old URL with the raw GitHub content URL
# old_url = 'https://projects.fivethirtyeight.com/soccer-api/club/spi_matches_latest.csv'

try:
    matches = pd.read_csv('https://raw.githubusercontent.com/walesiam/Walesiam/main/spi_matches_latest.csv')
except Exception as e:
    print(f"Error loading data from URL: {e}")
    print("Please check your internet connection or the validity of the URL.")
    # Initialize an empty DataFrame if loading fails, so the script doesn't immediately break.
    matches = pd.DataFrame(columns=['date','league', 'team1', 'team2', 'spi1', 'spi2', 'prob1', 'prob2', 'score1', 'score2'])


# Select relevant columns for analysis
if not matches.empty:
    matches_edit = matches[['date','league', 'team1', 'team2', 'spi1', 'spi2', 'prob1', 'prob2', 'score1', 'score2']]
else:
    matches_edit = pd.DataFrame(columns=['date','league', 'team1', 'team2', 'spi1', 'spi2', 'prob1', 'prob2', 'score1', 'score2'])


def team_league(x):
    # x: team name

    if matches_edit.empty and matches.empty: # Check if initial data load failed
        print(f"Cannot process team {x} because initial data loading failed or yielded no data.")
        return pd.DataFrame()

    # --- Infer the primary domestic league (y) for team x ---
    team_matches_all = matches_edit[(matches_edit['team1'] == x) | (matches_edit['team2'] == x)]

    if team_matches_all.empty:
        print(f"No matches found for team {x} in the dataset.")
        return pd.DataFrame()

    domestic_leagues_matches = team_matches_all[
        ~team_matches_all['league'].isin(['UEFA Champions League', 'UEFA Europa League', 'UEFA Europa Conference League'])
    ]

    y = None # Initialize y
    if domestic_leagues_matches.empty:
        print(f"No domestic league matches found for team {x}. They might only play in UEFA competitions or an unlisted league.")
        print(f"Warning: Could not infer a primary domestic league for {x}.")
    else:
        mode_result = domestic_leagues_matches['league'].mode()
        if not mode_result.empty:
            y = mode_result[0]
            print(f"Inferred domestic league for {x}: {y}")
        else:
            print(f"Could not determine a primary domestic league for {x} from its matches.")

    # Filter matches for the inferred domestic league (if found) and all major UEFA competitions
    conditions = (matches_edit['league'] == 'UEFA Champions League') | \
                 (matches_edit['league'] == 'UEFA Europa League') | \
                 (matches_edit['league'] == 'UEFA Europa Conference League')
    if y:
        conditions = conditions | (matches_edit['league'] == y)

    total = matches_edit.loc[conditions].copy() # Use .copy()

    if total.empty:
        print(f"No matches found for team {x} in inferred domestic league or UEFA competitions.")
        return pd.DataFrame()

    total[['outcome', 'points1', 'points2']] = 'unknown', pd.NA, pd.NA # Use pd.NA for numeric missing vals

    # Determine match outcomes based on scores (only for matches with non-null scores)
    valid_scores = total['score1'].notna() & total['score2'].notna()
    total.loc[valid_scores & (total['score1'] < total['score2']), 'outcome'] = 'away win'
    total.loc[valid_scores & (total['score1'] > total['score2']), 'outcome'] = 'home win'
    total.loc[valid_scores & (total['score1'] == total['score2']), 'outcome'] = 'draw'

    # Assign points based on match outcomes
    total.loc[total['outcome'] == 'home win', 'points1'] = 3
    total.loc[total['outcome'] == 'home win', 'points2'] = 0
    total.loc[total['outcome'] == 'away win', 'points2'] = 3
    total.loc[total['outcome'] == 'away win', 'points1'] = 0
    total.loc[total['outcome'] == 'draw', 'points1'] = 1
    total.loc[total['outcome'] == 'draw', 'points2'] = 1

    # Initialize total_points
    total_league_points = pd.NA # Use pd.NA for undefined total points

    # --- Calculate points and stats for the specified team (x) ---
    stats_summary = []

    # Domestic League Statistics
    if y:
        hp = total.loc[(total['league'] == y) & (total['team1'] == x) & (total['points1'] == 3), 'points1'].sum()
        ap = total.loc[(total['league'] == y) & (total['team2'] == x) & (total['points2'] == 3), 'points2'].sum()
        hd = total.loc[(total['league'] == y) & (total['team1'] == x) & (total['points1'] == 1), 'points1'].sum()
        ad = total.loc[(total['league'] == y) & (total['team2'] == x) & (total['points2'] == 1), 'points2'].sum()
        total_league_points = hp + ap + hd + ad

        league_stats_data = {
            'Statistics': ['Home Wins', 'Away Wins', 'Home Losses', 'Away Losses', 'Home Draws', 'Away Draws', 'Unplayed Matches'],
            'Count': [
                total.loc[(total['team1'] == x) & (total['outcome'] == 'home win') & (total['league'] == y), 'outcome'].count(),
                total.loc[(total['team2'] == x) & (total['outcome'] == 'away win') & (total['league'] == y), 'outcome'].count(),
                total.loc[(total['team1'] == x) & (total['outcome'] == 'away win') & (total['league'] == y), 'outcome'].count(),
                total.loc[(total['team2'] == x) & (total['outcome'] == 'home win') & (total['league'] == y), 'outcome'].count(),
                total.loc[(total['team1'] == x) & (total['outcome'] == 'draw') & (total['league'] == y), 'outcome'].count(),
                total.loc[(total['team2'] == x) & (total['outcome'] == 'draw') & (total['league'] == y), 'outcome'].count(),
                total.loc[((total['team1'] == x) | (total['team2'] == x)) & (total['outcome'] == 'unknown') & (total['league'] == y), 'outcome'].count()
            ]
        }
        league_stats_df = pd.DataFrame(league_stats_data)
        print(f"\n--- {y} Statistics for {x} ---")
        if not pd.isna(total_league_points):
             print(f"Total League Points: {total_league_points}")
        # print(league_stats_df.to_string(index=False))
        print(tabulate(league_stats_df, headers='keys', tablefmt='psql', showindex=False))
    else:
        print(f"\n--- Domestic League Statistics for {x} ---")
        print("Could not be calculated as domestic league was not inferred.")

    # UEFA Competitions Statistics
    competitions = {
        'UEFA Champions League': 'UCL',
        'UEFA Europa League': 'UEL',
        'UEFA Europa Conference League': 'ECL'
    }

    for comp_name, short_name in competitions.items():
        stats_data = {
            'Statistics': ['Home Wins', 'Away Wins', 'Home Losses', 'Away Losses', 'Home Draws', 'Away Draws', 'Unplayed Matches'],
            'Count': [
                total.loc[(total['team1'] == x) & (total['outcome'] == 'home win') & (total['league'] == comp_name), 'outcome'].count(),
                total.loc[(total['team2'] == x) & (total['outcome'] == 'away win') & (total['league'] == comp_name), 'outcome'].count(),
                total.loc[(total['team1'] == x) & (total['outcome'] == 'away win') & (total['league'] == comp_name), 'outcome'].count(),
                total.loc[(total['team2'] == x) & (total['outcome'] == 'home win') & (total['league'] == comp_name), 'outcome'].count(),
                total.loc[(total['team1'] == x) & (total['outcome'] == 'draw') & (total['league'] == comp_name), 'outcome'].count(),
                total.loc[(total['team2'] == x) & (total['outcome'] == 'draw') & (total['league'] == comp_name), 'outcome'].count(),
                total.loc[((total['team1'] == x) | (total['team2'] == x)) & (total['outcome'] == 'unknown') & (total['league'] == comp_name), 'outcome'].count()
            ]
        }
        stats_df = pd.DataFrame(stats_data)
        # Only print the table if there are any matches (played or unplayed) for this team in this competition
        if stats_df['Count'].sum() > 0 or \
           any(((total['team1'] == x) | (total['team2'] == x)) & (total['league'] == comp_name)):
            print(f"\n--- {comp_name} ({short_name}) Statistics for {x} ---")
            # print(stats_df.to_string(index=False))
            # Use tabulate here
            print(tabulate(stats_df, headers='keys', tablefmt='psql', showindex=False))

    # Return all matches involving the specified team (x) from the filtered leagues
    return total.loc[(total['team1'] == x) | (total['team2'] == x)]


In [145]:
team_league('Juventus')

Inferred domestic league for Juventus: Italy Serie A

--- Italy Serie A Statistics for Juventus ---
Total League Points: 59
+------------------+---------+
| Statistics       |   Count |
|------------------+---------|
| Home Wins        |       9 |
| Away Wins        |       8 |
| Home Losses      |       3 |
| Away Losses      |       2 |
| Home Draws       |       3 |
| Away Draws       |       5 |
| Unplayed Matches |       8 |
+------------------+---------+

--- UEFA Champions League (UCL) Statistics for Juventus ---
+------------------+---------+
| Statistics       |   Count |
|------------------+---------|
| Home Wins        |       3 |
| Away Wins        |       2 |
| Home Losses      |       1 |
| Away Losses      |       1 |
| Home Draws       |       0 |
| Away Draws       |       1 |
| Unplayed Matches |       0 |
+------------------+---------+


,date,league,team1,team2,spi1,spi2,prob1,prob2,score1,score2,outcome,points1,points2
1026,2021-08-22,Italy Serie A,Udinese,Juventus,55.71,81.81,0.1413,0.6587,2.0,2.0,draw,1,1
1217,2021-08-28,Italy Serie A,Juventus,Empoli,80.75,52.40,0.7452,0.0893,0.0,1.0,away win,0,3
1458,2021-09-11,Italy Serie A,Napoli,Juventus,78.50,79.90,0.3967,0.3562,2.0,1.0,home win,3,0
1613,2021-09-14,UEFA Champions League,Malmo FF,Juventus,48.42,79.07,0.1318,0.6841,0.0,3.0,away win,0,3
1866,2021-09-19,Italy Serie A,Juventus,AC Milan,79.60,78.73,0.4317,0.3249,1.0,1.0,draw,1,1
1932,2021-09-22,Italy Serie A,Spezia,Juventus,52.23,79.42,0.1464,0.6616,2.0,3.0,away win,0,3
2118,2021-09-26,Italy Serie A,Juventus,Sampdoria,79.16,62.17,0.6579,0.1392,3.0,2.0,home win,3,0
2262,2021-09-29,UEFA Champions League,Juventus,Chelsea,79.00,88.65,0.3279,0.4131,1.0,0.0,home win,3,0
2399,2021-10-02,Italy Serie A,Torino,Juventus,62.00,78.69,0.2037,0.5689,0.0,1.0,away win,0,3
2795,2021-10-17,Italy Serie A,Juventus,AS Roma,78.65,73.12,0.4889,0.2793,1.0,0.0,home win,3,0
